In [1]:
# 파이썬≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
import keras

# 깔끔한 그래프 출력을 위해 
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

mpl.rcParams['axes.unicode_minus'] = False

# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import seaborn as sns

import scipy as sp
from scipy import stats

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 노트북 실행 결과를 동일하게 유지하기 위해
# np.random.seed(42)

# LSTM 을 이용한 시계열 예측

## 데이터 읽어오기

In [2]:
data = pd.read_csv('data_ML/kakao.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5391 non-null   object 
 1   Open       5385 non-null   float64
 2   High       5385 non-null   float64
 3   Low        5385 non-null   float64
 4   Close      5385 non-null   float64
 5   Adj Close  5385 non-null   float64
 6   Volume     5385 non-null   float64
dtypes: float64(6), object(1)
memory usage: 294.9+ KB


## 데이터 전처리

In [3]:
# 결측치 제거
data = data.dropna(axis = 0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5385 entries, 0 to 5390
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5385 non-null   object 
 1   Open       5385 non-null   float64
 2   High       5385 non-null   float64
 3   Low        5385 non-null   float64
 4   Close      5385 non-null   float64
 5   Adj Close  5385 non-null   float64
 6   Volume     5385 non-null   float64
dtypes: float64(6), object(1)
memory usage: 336.6+ KB


In [4]:
# 피처를 파생 - High 와 Low 의 평균
high_price = data['High'].values
low_price = data['Low'].values
mid_price = (high_price + low_price) / 2

## timestep 을 50으로 설정해서 데이터를 생성

In [5]:
day_divided = 50
day_length = day_divided + 1
day_result = []

for i in range(len(mid_price) - day_length) :
    day_result.append(mid_price[i : i + day_length])
print('전체 데이터 Length:' , len(data))
print('나눈 데이터 Length:' , len(day_result))
print(day_result[0])
print(day_result[1])

전체 데이터 Length: 5385
나눈 데이터 Length: 5334
[54600. 53850. 47650. 44350. 50250. 48150. 42950. 41300. 36550. 39150.
 41950. 38850. 37350. 36800. 36800. 32300. 30950. 29400. 31400. 33400.
 36500. 33600. 29600. 28800. 29700. 27450. 23900. 21800. 23600. 22900.
 22100. 22450. 23350. 21600. 20600. 22400. 19920. 17910. 17560. 18380.
 19190. 21750. 21650. 19400. 19400. 17740. 15040. 14910. 14100. 12830.
 13170.]
[53850. 47650. 44350. 50250. 48150. 42950. 41300. 36550. 39150. 41950.
 38850. 37350. 36800. 36800. 32300. 30950. 29400. 31400. 33400. 36500.
 33600. 29600. 28800. 29700. 27450. 23900. 21800. 23600. 22900. 22100.
 22450. 23350. 21600. 20600. 22400. 19920. 17910. 17560. 18380. 19190.
 21750. 21650. 19400. 19400. 17740. 15040. 14910. 14100. 12830. 13170.
 13380.]


In [6]:
# 정규화 수행, 배열로 변환
norm_result = []

# 첫 번째 데이터를 기준으로 정규화를 수행
# 첫 번째 데이터를 0으로 해서 나머지 데이터의 비율을 생성
for section in day_result :
    norm_section = [((float(p)) / (float(section[0]))-1) for p in section]
    norm_result.append(norm_section)
day_result = np.array(norm_result)

print(day_result[0])


[ 0.    -0.014 -0.127 -0.188 -0.08  -0.118 -0.213 -0.244 -0.331 -0.283
 -0.232 -0.288 -0.316 -0.326 -0.326 -0.408 -0.433 -0.462 -0.425 -0.388
 -0.332 -0.385 -0.458 -0.473 -0.456 -0.497 -0.562 -0.601 -0.568 -0.581
 -0.595 -0.589 -0.572 -0.604 -0.623 -0.59  -0.635 -0.672 -0.678 -0.663
 -0.649 -0.602 -0.603 -0.645 -0.645 -0.675 -0.725 -0.727 -0.742 -0.765
 -0.759]


In [7]:
# 훈련 데이터와 테스트 데이터 분할

# 훈련 데이터 비율 설정
train_data_rate = 0.9

# 경계값 인덱스 구하기
boundary = round(day_result.shape[0] * train_data_rate)

# 경계값 인덱스를 기준으로 훈련, 검증 데이터 분할
train_data = day_result[:boundary, :]
test_data = day_result[boundary:, :]

print(train_data.shape)
print(test_data.shape)

# 훈련데이터 타겟 설정
X_train = train_data[:, : -1]

# 딥러닝에서는 데이터의 차원이 3차원이나 4차원으로 만들어져야 합니다.
# CNN 에서는 4차원 데이터(샘플 개수, 높이, 너비, 채널)
# RNN 에서는 3차원 데이터(샘플 개수, 높이, 너비)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

y_train = train_data[:,  -1]
print(X_train.shape)
print(y_train.shape)

# 검증 데이터 타겟 설정
X_test = test_data[:, : -1]

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_test = test_data[:, -1]
print(X_test.shape)
print(y_test.shape)

(4801, 51)
(533, 51)
(4801, 50, 1)
(4801,)
(533, 50, 1)
(533,)


# 모델 생성 및 훈련

In [11]:
import keras

model = keras.models.Sequential()

# RNN 층 이전 층은 출력을 전달을 해줘야 합니다.
#return_sequences 를 True로 설정하면 됩니다.
model.add(keras.layers.LSTM(50, return_sequences = True, input_shape = (50, 1)))
# model.add(keras.layers.LSTM(64, return_sequences = True))
model.add(keras.layers.LSTM(64, return_sequences = True))
model.add(keras.layers.Dense(1, activation = 'relu'))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 50, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 50, 64)         │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50, 1)          │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,905 (155.88 KB)

 Trainable params: 39,905 (155.88 KB)

 Non-trainable params: 0 (0.00 B)

# 모델 컴파일 하기

In [12]:
model.compile(loss = 'mse', optimizer = 'Adam')
model.fit(X_train, y_train, validation_data = (X_test, y_test),
         batch_size = 10, epochs = 15)

Epoch 1/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.0567 - val_loss: 0.0829
Epoch 2/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0519 - val_loss: 0.0829
Epoch 3/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0541 - val_loss: 0.0829
Epoch 4/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0523 - val_loss: 0.0829
Epoch 5/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0522 - val_loss: 0.0829
Epoch 6/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0565 - val_loss: 0.0829
Epoch 7/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0560 - val_loss: 0.0829
Epoch 8/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0520 - val_loss: 0.0829
Epoch 9/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0520 - val_loss: 0.0829
Epoch 10/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0549 - val_loss: 0.0829
Epoch 11/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0560 - val_loss: 0.0829
Epoch 12/15
481/481 ━━━━━━━━━━━━━━━━━━━━

In [13]:
# 예측
pred = model.predict(X_test)
print(pred)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]


# GRU 이용하기

In [14]:
import keras

model = keras.models.Sequential()

# RNN 층 이전 층은 출력을 전달을 해줘야 합니다.
#return_sequences 를 True로 설정하면 됩니다.
# LSTM 은 입력, 출력, 망각게이트 3개를 가지지만
# GRU는 업데이트와 리셋이라는 2개의 게이트만 소유
# GRU 를 LSTM 의 간소화된 버전이라고 합니다.
model.add(keras.layers.GRU(50, return_sequences = True, input_shape = (50, 1)))
model.add(keras.layers.GRU(64, return_sequences = True))
model.add(keras.layers.GRU(64, return_sequences = True))
model.add(keras.layers.Dense(1, activation = 'relu'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 50, 50)         │         7,950 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 50, 64)         │        22,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 50, 64)         │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50, 1)          │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,247 (215.81 KB)

 Trainable params: 55,247 (215.81 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(loss = 'mse', optimizer = 'sgd')
model.fit(X_train, y_train, validation_data = (X_test, y_test),
         batch_size = 10, epochs = 15)

Epoch 1/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - loss: 0.0520 - val_loss: 0.0678
Epoch 2/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 0.0544 - val_loss: 0.0719
Epoch 3/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 0.0563 - val_loss: 0.0569
Epoch 4/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - loss: 0.0541 - val_loss: 0.0633
Epoch 5/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - loss: 0.0535 - val_loss: 0.0660
Epoch 6/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - loss: 0.0547 - val_loss: 0.0630
Epoch 7/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - loss: 0.0546 - val_loss: 0.0725
Epoch 8/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - loss: 0.0536 - val_loss: 0.0529
Epoch 9/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - loss: 0.0585 - val_loss: 0.0709
Epoch 10/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - loss: 0.0521 - val_loss: 0.0690
Epoch 11/15
481/481 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0537 - val_loss: 0.0730
Epoch 12/15
481/481 ━━━━━━━━━━

In [16]:
# 예측
pred = model.predict(X_test)
print(pred)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
[[[0.03 ]
  [0.034]
  [0.035]
  ...
  [0.054]
  [0.054]
  [0.053]]

 [[0.03 ]
  [0.034]
  [0.035]
  ...
  [0.053]
  [0.053]
  [0.052]]

 [[0.03 ]
  [0.034]
  [0.035]
  ...
  [0.055]
  [0.055]
  [0.054]]

 ...

 [[0.03 ]
  [0.034]
  [0.035]
  ...
  [0.078]
  [0.078]
  [0.078]]

 [[0.03 ]
  [0.034]
  [0.035]
  ...
  [0.078]
  [0.078]
  [0.077]]

 [[0.03 ]
  [0.034]
  [0.035]
  ...
  [0.077]
  [0.077]
  [0.076]]]


# 자연어 처리

## 토큰화 및 문장 인코딩

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    '희찬이는 나를 정말 정말 좋아해',
    '희찬이는 영화를 무척 좋아해'
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

{'희찬이는': 1, '정말': 2, '좋아해': 3, '나를': 4, '영화를': 5, '무척': 6}


In [20]:
# 문장 인코딩
word_encoding = tokenizer.texts_to_sequences(sentences)
print(word_encoding)

[[1, 4, 2, 2, 3], [1, 5, 6, 3]]


## OOV - 사전에 없는 단어

In [21]:
new_sequences = ['희찬이는 별다줄을 좋아해']

# 사전에 없는 단어는 기본적으로 무시가 됨
word_encoding = tokenizer.texts_to_sequences(new_sequences)
print(word_encoding)

[[1, 3]]


In [23]:
tokenizer = Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

word_encoding = tokenizer.texts_to_sequences(new_sequences)
print(word_encoding)

{'<OOV>': 1, '희찬이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6, '무척': 7}
[[2, 1, 4]]


In [24]:
tokenizer = Tokenizer(oov_token = '<OOV>', num_words = 3)
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

# 문장을 인코딩할 때 3개의 단어만 사용하고 나머지는 모두 OOV
word_encoding = tokenizer.texts_to_sequences(new_sequences)
print(word_encoding)

{'<OOV>': 1, '희찬이는': 2, '정말': 3, '좋아해': 4, '나를': 5, '영화를': 6, '무척': 7}
[[2, 1, 1]]


## 패딩 - 길이 맞추기

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    '희찬이는 나를 정말 정말 좋아해',
    '희찬이는 영화를 무척 좋아해'
]
word_encoding =  tokenizer.texts_to_sequences(sentences)

#2개의 문장 길이를 동일하게 맞추기
print(pad_sequences(word_encoding))
print(pad_sequences(word_encoding, padding = 'post'))

[[2 1 1 1 1]
 [0 2 1 1 1]]
[[2 1 1 1 1]
 [2 1 1 1 0]]
